*Note this workflow task should be executed against a Serverless SQL Warehouse* 

# FHIR Bundle Meta 
***

In [0]:
DECLARE OR REPLACE VARIABLE catalog_use STRING DEFAULT 'fhir_workshop';
DECLARE OR REPLACE VARIABLE schema_use STRING DEFAULT REPLACE(SPLIT(current_user(), '@')[0], '.', '_');

In [0]:
SET VARIABLE catalog_use = :catalog_use; 
SET VARIABLE schema_use = :schema_use;  

In [0]:
USE IDENTIFIER(catalog_use || "." || schema_use);

In [0]:
select current_catalog(), current_schema();

In [0]:
CREATE OR REFRESH STREAMING TABLE bundle_meta (
  bundle_uuid STRING PRIMARY KEY COMMENT 'Unique identifier for the FHIR bundle and the primary key for the table.  This column will be required for all joins amongst resource tables as Resource primary keys are only guaranteed to be unique inside of a bundle.'
  ,file_metadata STRUCT<
    file_path: STRING
    ,file_name: STRING
    ,file_size: BIGINT
    ,file_block_start: BIGINT
    ,file_block_length: BIGINT
    ,file_modification_time: TIMESTAMP
  > COMMENT 'Original meta date of the file ingested from the volume.'
  ,ingest_time TIMESTAMP COMMENT 'The date timestamp the file was ingested.'
  ,bundle_resourceType STRING COMMENT "The FHIR Bundle's overall resource type."
  ,bundle_type STRING COMMENT "The FHIR Bundle's overall type."
  ,meta VARIANT COMMENT 'Metadata about the FHIR Bundle overall, typically includes information about the source of the bundle, but is not required to be passed for a valid HL7 transaction.  Always NULL for data that originated from Synthea.'
)
COMMENT 'Original FHIR Bundle Metadata'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
  ,'delta.enableDeletionVectors' = 'true'
  ,'delta.enableRowTracking' = 'true'
  ,'quality' = 'bronze'
  ,'pipelines.channel' = 'PREVIEW'
  ,'delta.feature.variantType-preview' = 'supported'
)
AS SELECT
  bundle_uuid
  ,file_metadata
  ,ingest_time
  ,fhir:resourceType::string as bundle_resourceType
  ,fhir:type::string as bundle_type
  ,fhir:Meta as meta
FROM 
  STREAM (fhir_bronze_variant);